In [10]:
import subprocess # Library for using the terminal
import os
from dotenv import load_dotenv
import json
from constants import *
import bit
import web3
from web3 import Web3
from bit.network import NetworkAPI
from eth_account import Account
from web3.middleware import geth_poa_middleware

In [11]:
# Load .env enviroment variables and mnemomic
load_dotenv()
mnemonic = os.getenv('MNEMONIC')

In [14]:
# Function used to derive wallet keys from master key
def derive_wallets(mnemonic,Coin,Numderive):

    # For Windows users the php derive part replaces ./derive
    command = f'php derive -g --mnemonic="{mnemonic}" --cols=address,index,path,privkey,pubkey,pubkeyhash,xprv,xpub --coin={Coin} --numderive={Numderive} --format=json'

    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, error) = p.communicate()
    p_status = p.wait()

    # print(output)
    keys = json.loads(output)
    return keys

coins = {}
types = [BTCTEST,ETH]

for type in types:
    coins[type] = derive_wallets(Mnemonic, type, 3)

coins

{'btc-test': [{'address': 'muxWWxbTF8vpQsyMmTunmHWfognKL89M1o',
   'index': 0,
   'path': "m/44'/1'/0'/0/0",
   'privkey': 'cUq1x97zQDK9afFh9g1dPFA2kNv9FtvnmqEd1C3H2MNU5zyxfmuY',
   'pubkey': '037e0a997a575c6d4224ed3ea991886a8823e7240da1497169337438055edc4127',
   'pubkeyhash': '9e670ed9c78811f7a6ca79be01c166f0d98fedae',
   'xprv': 'tprv8kQQaLPfW8ccxhzzagrLy6yru3RDPmehBv5sN97EUoRfAgqmevdTcMmj6yfYcJ3JpTJrUuA2S5gjFzQLTiC8ojxTMtAZTQdQ4i3TwrYuCEH',
   'xpub': 'tpubDH6SikRueWJHrB2nULWwNWdyU4w9Z6qbmDgeef9Xu5E41B6YHKT3nrPbH7sU6YzvYyTXga6q3AXp6BQSho4niNe6XdKBtGA9EEaqDy4JjPU'},
  {'address': 'mjc1nEPqXXC5NdwcnJ7DypBUSjSPNWCmn3',
   'index': 1,
   'path': "m/44'/1'/0'/0/1",
   'privkey': 'cRDpmh4S3yft1isH3K6nShAoLDF6iLczr4jk5vQACzsAwtL5Yu8R',
   'pubkey': '0226efcea7405c91ea6c5800b561d06fedc4b88208cab596d64a700f525230d568',
   'pubkeyhash': '2cd57e25ad07bc5f336e182b79aa0219eb8db41c',
   'xprv': 'tprv8kQQaLPfW8cd1jcSxkei1PyBvBFcHCNCTKG5sNNJAdT5dFQ6VPpbcpMsKqVBh4S6i71v3dyucRYpeHcvFRBQoJXnaEtzoWUfi

In [21]:
coins[BTCTEST][0]['address']

'muxWWxbTF8vpQsyMmTunmHWfognKL89M1o'

In [18]:
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

# Function used to convert key to account
def priv_key_to_account(coin,priv_key):
    if coin == BTCTEST:
        return bit.PrivateKeyTestnet(priv_key)
    elif coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    else:
        print('Must use either BTCTEST or ETH')

# Function used to create a transaction
def create_tx(coin,account,to,amount):
    if coin == BTCTEST:
        return bit.PrivateKeyTestnet.prepare_transaction(account.address, [(to,amount, BTC)])
    elif coin == ETH:
        
        
        gas_estimate = w3.eth.estimateGas(
            {'from': account.address, 'to': to, 'value': amount}
        )

        return {
            'from': account.address,
            'to': to,
            'value': amount,
            'gasPrice': w3.eth.gasPrice,
            'gas': gas_estimate,
            'nonce': w3.eth.getTransactionCount(account.address)
        }
    else:
        print('Must use either BTCTEST or ETH')

# Function used to send transaction
def send_tx(coin,account,to,amount):
    if coin == BTCTEST:
        raw_tx = create_tx(coin,account,to,amount)
        signed = account.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(signed)
    elif coin == ETH:
        raw_tx = create_tx(coin,account,to,amount)
        signed = account.signTransaction(raw_tx)
        return w3.eth.sendRawTransaction(signed.rawTransaction)
    else:
        print('Must use either BTCTEST or ETH')   

# Sending a Transaction

In [20]:
# Setting variables to pass functions
# ETH and BTCTEST are the only supported currencies and do not need strings as they are global variables
currency_to_transact = BTCTEST
owner_account_priv_key = coins[currency_to_transact][0]['privkey']
receiver_account = coins[currency_to_transact][1]['address']
ETH_to_send = 10000
BTCTEST_to_send = 0.00001

# Set the account key with the appropriate package using the above function
account = priv_key_to_account(currency_to_transact,owner_account_priv_key)

# Use if statement to pass the right amount to the function for sending a transaction
if currency_to_transact == ETH:
    send_tx(currency_to_transact,account,receiver_account,ETH_to_send)
elif currency_to_transact == BTCTEST:
    send_tx(currency_to_transact,account,receiver_account,BTCTEST_to_send)
else:
    print('You can only use ETH or BTCTEST to transact.')